In [1]:
import chess.pgn
import tensorflow as tf
from tensorflow.keras.layers import Embedding, LSTM, Dense, Conv1D, MaxPooling1D, Flatten, Dropout, LeakyReLU
from tensorflow.keras.models import Sequential, load_model
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
import chess
from tensorflow.keras.callbacks import ModelCheckpoint, Callback
from tensorflow.keras import backend as K
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import pandas as pd
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt


2024-08-13 17:35:24.339948: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-08-13 17:35:24.606749: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-13 17:35:26.620984: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-08-13 17:35:28.867458: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-08-13 17:35:30.165355: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been 

In [ ]:
data = pd.read_csv('data/games.csv')

# Dict to convert UCI -> Int and Int -> UCI
uci_to_int = {}
int_to_uci = {}
counter = 1 

# Convert UCI -> Int Move
def move_to_int(move):
    if move not in uci_to_int:
        global counter
        uci_to_int[move] = counter
        int_to_uci[counter] = move
        counter += 1
    return uci_to_int[move]

# Convert a string of moves into ints using the chess library
def parse_moves(moves_str):
    board = chess.Board()
    move_list = []
    for move in moves_str.split():
        try:
            uci_move = board.push_san(move).uci()
            move_list.append(move_to_int(uci_move))
        except ValueError:
            print(f"Invalid move: {move}")
            break
    return move_list


data['parsed_moves'] = data['moves'].apply(parse_moves)

X = pad_sequences(data['parsed_moves'], maxlen=28, padding='post', truncating='post')

## Finding out how many moves we have (I think 27)
max_index = max(uci_to_int.values())


In [ ]:
## To avoid alot of the same name ones we cluster into just those with the same name rather than the eco
grouped_eco_labels = {
    'A00': 'Polish (Sokolsky) opening',
    'A01': 'Nimzovich-Larsen attack',
    'A02-A03': "Bird's opening",
    'A04-A09': 'Reti opening',
    'A10-A39': 'English opening',
    'A40-A44': "Queen's pawn",
    'A45-A46': "Queen's pawn game",
    'A47': "Queen's Indian defence",
    'A48-A49': "King's Indian defence",
    'A50': "Queen's pawn game",
    'A51-A52': 'Budapest defence',
    'A53-A55': 'Old Indian defence',
    'A56': 'Benoni defence',
    'A57-A59': 'Benko gambit',
    'A60-A79': 'Benoni defence',
    'A80-A99': 'Dutch',
    'B00': "King's pawn opening",
    'B01': 'Scandinavian (centre counter) defence',
    'B02-B05': "Alekhine's defence",
    'B06': 'Robatsch (modern) defence',
    'B07-B09': 'Pirc defence',
    'B10-B19': 'Caro-Kann defence',
    'B20-B99': 'Sicilian defence',
    'C00-C19': 'French defence',
    'C20-C99': "King's pawn game",
    'D00-D99': "Queen's Gambit",
    'E00-E99': "King's Indian defence",
}


## Converting the eco to integers
eco_to_int = {}
int_to_opening = {}
counter = 0

for eco_range, opening_name in grouped_eco_labels.items():
    if '-' in eco_range:
        start, end = eco_range.split('-')
        for i in range(int(start[1:]), int(end[1:]) + 1):
            eco = start[0] + str(i).zfill(2)
            if eco not in eco_to_int:
                eco_to_int[eco] = counter
    else:
        if eco_range not in eco_to_int:
            eco_to_int[eco_range] = counter

    int_to_opening[counter] = opening_name
    counter += 1

data['opening_encoded'] = data['opening_eco'].map(eco_to_int)

print(data[['opening_eco', 'opening_encoded']].head())

num_unique_openings = data['opening_encoded'].nunique()

print(f"Number of unique encoded openings: {num_unique_openings}")


  opening_eco  opening_encoded
0         D10               25
1         B00               16
2         C20               24
3         D02               25
4         C41               24
Number of unique encoded openings: 27


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, data['opening_encoded'], test_size=0.2, random_state=42)

print(X_train.shape)

(16046, 28)


In [ ]:
## Prints the validation progress to a png file every "save_every" epochs
class ValidationLossPlotter(Callback):
    def __init__(self, save_every=5):
        super(ValidationLossPlotter, self).__init__()
        self.epoch_count = 0
        self.save_every = save_every
        self.history = []

    def on_epoch_end(self, epoch, logs=None):
        val_loss = logs.get('val_loss')
        self.history.append(val_loss)
        self.epoch_count += 1

        plt.figure(figsize=(10, 6))
        plt.plot(range(1, self.epoch_count + 1), self.history, label='Validation Loss')
        plt.xlabel('Epochs')
        plt.ylabel('Validation Loss')
        plt.title('Validation Loss Progress')
        plt.legend()

        if self.epoch_count % self.save_every == 0:
            plt.savefig('validation_progress.png')

        plt.close()

In [ ]:
## Simple lstm model
model = Sequential([
    Embedding(input_dim=max_index + 1, output_dim=128, input_length=100),  
    LSTM(256, return_sequences=True),
    LSTM(128, return_sequences=True),  
    LSTM(64),  
    Dense(128, activation='relu'), 
    Dense(num_unique_openings, activation='softmax') 
])

## Setting up checkpoint and validation plotter
best_model_name = 'best_model.keras'
validation_plotter = ValidationLossPlotter(save_every=2)
checkpoint = ModelCheckpoint(best_model_name, monitor='val_accuracy', save_best_only=True, mode='max', verbose=1)

## Compile
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

## Run
epochs = 25
model.fit(X_train, y_train, epochs=epochs, batch_size=1024, validation_split=0.2, shuffle=True, 
          callbacks=[checkpoint, validation_plotter])

/mnt/c/Users/Maxwell Bruce/Desktop/AML/ChessOpenningMovePredictor/myenv/lib/python3.10/site-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 839ms/step - accuracy: 0.2474 - loss: 3.0785
Epoch 1: val_accuracy improved from -inf to 0.31402, saving model to best_model.keras
13/13 ━━━━━━━━━━━━━━━━━━━━ 18s 1s/step - accuracy: 0.2503 - loss: 3.0615 - val_accuracy: 0.3140 - val_loss: 2.4429
Epoch 2/2
13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 777ms/step - accuracy: 0.3071 - loss: 2.4458
Epoch 2: val_accuracy did not improve from 0.31402
13/13 ━━━━━━━━━━━━━━━━━━━━ 12s 902ms/step - accuracy: 0.3073 - loss: 2.4446 - val_accuracy: 0.3140 - val_loss: 2.3871


In [ ]:

## Load the best model and print metrics
best_model = tf.keras.models.load_model('best_model.keras')

y_pred = best_model.predict(X_test)
y_pred_classes = y_pred.argmax(axis=1)

accuracy = accuracy_score(y_test, y_pred_classes)
precision = precision_score(y_test, y_pred_classes, average='weighted', zero_division=1)
recall = recall_score(y_test, y_pred_classes, average='weighted')
f1 = f1_score(y_test, y_pred_classes, average='weighted', zero_division=1)

print(f"Test Accuracy: {accuracy}")
print(f"Test Precision: {precision}")
print(f"Test Recall: {recall}")
print(f"Test F1-Score: {f1}")

126/126 ━━━━━━━━━━━━━━━━━━━━ 3s 26ms/step
Test Accuracy: 0.2978564307078764
Test Precision: 0.7908620226061596
Test Recall: 0.2978564307078764
Test F1-Score: 0.13671535805489238


In [ ]:
## Now do the same except for random forest
X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

rf_model = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_model.fit(X_train_flat, y_train)

y_pred_rf = rf_model.predict(X_test_flat)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted', zero_division=1)
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted', zero_division=1)

print(f"RandomForest Test Accuracy: {accuracy_rf}")
print(f"RandomForest Test Precision: {precision_rf}")
print(f"RandomForest Test Recall: {recall_rf}")
print(f"RandomForest Test F1-Score: {f1_rf}")

RandomForest Test Accuracy: 0.9219840478564307
RandomForest Test Precision: 0.9242640467146178
RandomForest Test Recall: 0.9219840478564307
RandomForest Test F1-Score: 0.918758554948514
